In [8]:
from gensim.models.doc2vec import TaggedDocument

sentence = TaggedDocument(words=[u'some', u'words', u'here'], tags=[u'SENT_1'])

print(sentence)

TaggedDocument<['some', 'words', 'here'], ['SENT_1']>


In [9]:
import os
import gensim

test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 
                'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

In [19]:
%pip install smart-open
import smart_open

def read_corpus(file_name, tokens_only=False):
    with smart_open.smart_open(file_name, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(
                gensim.utils.simple_preprocess(line), [i])

Note: you may need to restart the kernel to use updated packages.


In [33]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [21]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

In [22]:
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, 
            epochs=model.epochs)

In [27]:
from gensim.models import Doc2Vec

models = [
    # PV-DBOW
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=10, epochs=50),

    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=10, epochs=50),
]

In [34]:
# A variável documents pode ser qualquer documento "taggeado", aqui vamos usar o train_corpus
documents = train_corpus

models[0].build_vocab(documents)
models[1].reset_from(models[0])

for model in models:
    model.train(documents, total_examples=model.corpus_count, 
                epochs=model.epochs)

In [39]:
# Comparando os dois modelos
%pip install testfixtures

from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

new_model = ConcatenatedDoc2Vec((models[0], models[1]))

     |████████████████████████████████| 95 kB 1.3 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.
